In [ ]:
import random
from tqdm import tqdm#进度条设置
from random import randint
import numpy as np
from os import urandom
from  keras. callbacks  import  ModelCheckpoint, LearningRateScheduler
from  keras. models  import  Model
from  keras. layers  import  Dense,  Conv2D,Input,Reshape, Permute,  Flatten, BatchNormalization, Activation, Dropout,DepthwiseConv2D
from keras.regularizers import l2
from  keras  import  layers
import tensorflow as tf
import copy

In [ ]:
def MBConvBlock(inputs, expansion_ratio, stride, filters, kernel_size):
    # 定义扩展通道数
    expand_channels = expansion_ratio * inputs.shape[-1]
    
    # 第一层扩展卷积
    x = Conv2D(expand_channels, kernel_size=(1, 1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 深度可分离卷积
    x = DepthwiseConv2D(kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x=Dropout(0.5)(x)
    # 线性投影回归
    x = Conv2D(filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
#     x = tf.keras.layers.Add()([x, inputs])
    
    return x

In [ ]:
bs = 2000
def cyclic_lr(epoch, high_lr):
    if epoch<=10:
        return 0.001 +  (0.0001*(10-epoch))
    elif 10<epoch<=25:
        return 0.001
    else:
        res=0.001 -  (0.0001*(epoch-25))
        if res>0.0001:
            return res
        return 0.0001
def make_resnet(num_words=16,multiset=16, num_filters=16, num_outputs=1, d1=1024, d2=1024, word_size=4, ks=3,depth=5, reg_param=0.00001, final_activation='sigmoid'):
  #Input and preprocessing layers
  inp = Input(shape=(1*num_words * word_size *multiset ,))
  rs = Reshape((4, 16,16))(inp)
  perm = Permute((2,3,1))(rs)
  input = Conv2D(num_filters , kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm)
  input = BatchNormalization()(input)
  input = Activation('relu')(input)
#densenet稠密神经网络
  for i in range(depth):
        conv1 = MBConvBlock(input, expansion_ratio=4, stride=1, filters=input.shape[-1], kernel_size=ks)
        conv2 = Conv2D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        input = tf.keras.layers.Concatenate()([conv2, input]) 
    
  flat1 = Flatten()(input)
  dense1 = Dense(1024,kernel_regularizer=l2(reg_param))(flat1)
  dense1 = BatchNormalization()(dense1)
  dense1 = Activation('relu')(dense1)
  dense2 = Dense(1024, kernel_regularizer=l2(reg_param))(dense1)
  dense2 = BatchNormalization()(dense2)
  dense2 = Activation('relu')(dense2)
  dense2=Dropout(0.5)(dense2)
  out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2)
  model = Model(inputs=inp, outputs=out)
  return(model)

In [ ]:
Sbox = np.array([0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2])
raw_P=[ 0,  8, 16, 24,  1,  9, 17, 25,  2, 10, 18, 26,  3, 11, 19, 27,  4,
       12, 20, 28,  5, 13, 21, 29,  6, 14, 22, 30,  7, 15, 23, 31]
raw_P = np.array(raw_P)
index = np.array([31 - i for i in range(32)])
raw_P = 31 - raw_P[index]
P = np.array([np.where(raw_P == i) for i in range(32)])
P = np.squeeze(P)

Sbox_inverse = np.array([0x5, 0xe, 0xf, 0x8, 0xc, 0x1, 0x2, 0xd, 0xb, 0x4, 0x6, 0x3, 0x0, 0x7, 0x9, 0xa])
P_inverse = raw_P

# for updating keys
KP = np.array([(i+61) % 80 for i in range(80)])

def get_Sbox_output_enc(x):
    n, m = np.shape(x)
    assert m == 4
    x_val = x[:, 0] * 8 + x[:, 1] * 4 + x[:, 2] * 2 + x[:, 3]
    y_val = Sbox[x_val]
    output = np.zeros((n, 4), dtype=np.uint8)
    for i in range(4):
        output[:, i] = (y_val >> (3 - i)) & 1
    # print('y_val shape is ', np.shape(y_val))
    return output

def get_Sbox_output_dec(x):
    n, m = np.shape(x)
    assert m == 4
    x_val = x[:, 0] * 8 + x[:, 1] * 4 + x[:, 2] * 2 + x[:, 3]
    y_val = Sbox_inverse[x_val]
    output = np.zeros((n, 4), dtype=np.uint8)
    for i in range(4):
        output[:, i] = (y_val >> (3 - i)) & 1
    # print('y_val shape is ', np.shape(y_val))
    return output

def update_master_key(keys, round_counter):
    tp = keys[:, KP]
    new_keys = copy.deepcopy(tp)
    new_keys[:, :4] = get_Sbox_output_enc(tp[:, :4])
    round_counter_arr = np.array([(round_counter >> (4-i)) & 1 for i in range(5) ], dtype=np.uint8)
    new_keys[:, 60:65] = tp[:, 60:65] ^ round_counter_arr
    return new_keys
#64位的密钥扩展
def expand_key(keys, nr):
    n, m = np.shape(keys)
    assert m == 80
    ks = np.zeros((nr+1, n, 64), dtype=np.uint8)
    ks[0] = keys[:, :64]
    for i in range(1, nr+1):
        keys = update_master_key(keys, i)
        ks[i] = keys[:, :64]
    #只要最右边的密钥
    return ks[:,:,32:64]

def sBoxLayer_enc(x):
    n, m = np.shape(x)
    assert m == 32
    output = np.zeros((n, 32), dtype=np.uint8)
    #8代表8个4位
    for i in range(8):
        st = 4 * i
        output[:, st:st+4] = get_Sbox_output_enc(x[:, st:st+4])
    return output

def sBoxLayer_dec(x):
    n, m = np.shape(x)
    assert m == 32
    output = np.zeros((n, 32), dtype=np.uint8)
    for i in range(8):
        st = 4 * i
        output[:, st:st+4] = get_Sbox_output_dec(x[:, st:st+4])
    return output

def pLayer_enc(x):
    output = x[:, P]
    return output

def pLayer_dec(x):
    output = x[:, P_inverse]
    return output

def enc_one_round(x, subkeys):
    y = sBoxLayer_enc(x)
    z = pLayer_enc(y)
    output = z ^ subkeys
    return output
def dec_one_round(x, subkeys):
    y = pLayer_dec(x)
    z = sBoxLayer_dec(y)
    output = z ^ subkeys
    return output
def encrypt(x, ks):
    nr = ks.shape[0]
    y = x ^ ks[0]
    for i in range(1, nr): 
        y = enc_one_round(y, ks[i])
    return y
def decrypt(x, ks):
    nr = ks.shape[0]
    y = x ^ ks[nr-1]
    for i in range(1, nr):
        y = dec_one_round(y, ks[nr - 1 - i])
    return y
def decrypt_1(x):
    y = pLayer_dec(x)
    z = sBoxLayer_dec(y)
    return z
#正常解密一轮
def decrypt_3(x):
    y = pLayer_dec(x)
    z = sBoxLayer_dec(y)
    return z
#生成实验数据
def decrypt_2(x,keys):
    y = x ^ keys
    y = pLayer_dec(y)
    z = sBoxLayer_dec(y)
    return z

In [ ]:
def make_recover_data(n,subbkey):
    x0 = np.frombuffer(urandom(n * 8), dtype=np.uint64)  # .reshape(-1, 1)
    p0 = np.zeros((n, 32), dtype=np.uint8)
    for i in range(32):
        off = 31 - i
        p0[:, i] = (x0 >> off) & 1
    arr = [[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0],
           [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1],
           [1, 1, 1, 0], [1, 1, 1, 1]]
    # master_keys = np.frombuffer(urandom(1 * 80), dtype=np.uint8).reshape(-1, 80) & 1
    subkeys = subbkey
    c0 = encrypt(p0, subkeys)
    pp_1=p0.copy()
    for j in range(4):
        pp_1[:,j+28]=pp_1[:,j+28]^arr[0][j]
    c0_1 = encrypt(pp_1, subkeys)
    creal=c0.copy()
    creal=np.concatenate((creal,c0_1),axis=1)
    for i in range(1,14,2):
        pp=p0.copy()
        pp_1=p0.copy()
        for j in range(4):
            pp[:,j+28]=pp[:,j+28]^arr[i][j]
            pp_1[:,j+28]=pp_1[:,j+28]^arr[i+1][j]
        c1=encrypt(pp, subkeys)
        c1_1=encrypt(pp_1, subkeys)
        creal=np.concatenate((creal,c1,c1_1),axis=1)
    return creal
def make_double_data(n,subbkey):
    X=make_recover_data(n,subbkey)
    X_1=make_recover_data(n,subbkey)
    X_data=np.concatenate((X,X_1),axis=1)
    return X_data

In [ ]:
def dec_on_subkeys(subbkey,left_key,right_key):
    #随机生成明文
    X=make_recover_data(1,subbkey)
    X_1=make_recover_data(1,subbkey)
    sub_keys=np.arange(left_key,right_key,dtype=np.uint64)
    #生成候选子密钥二进制
    sub_keys_bit = np.zeros((right_key-left_key, 32), dtype=np.uint8)
    for j in range(32):
        off = 31 - j
        sub_keys_bit[:, j] = (sub_keys >> off) & 1
    #候选子密钥解密
    cipher_1=decrypt_2(X[:,0:32],sub_keys_bit)
    cipher_2=decrypt_2(X_1[:,0:32],sub_keys_bit)
    for i in range(1,16):
        cipher_dec_1=decrypt_2(X[:,i*32:i*32+32],sub_keys_bit)
        cipher_dec_2=decrypt_2(X_1[:,i*32:i*32+32],sub_keys_bit)
        #我们的数据预处理方法
        cipher_dec_1=cipher_dec_1^cipher_1[:,0:32]
        cipher_dec_2=cipher_dec_2^cipher_2[:,0:32]
        cipher_1=np.concatenate((cipher_1,decrypt_3(cipher_dec_1)),axis=1)
        cipher_2=np.concatenate((cipher_2,decrypt_3(cipher_dec_2)),axis=1)
    cipher_1[:,0:32]=decrypt_3(cipher_1[:,0:32])
    cipher_2[:,0:32]=decrypt_3(cipher_2[:,0:32])
    return np.concatenate((cipher_1,cipher_2),axis=1)

In [ ]:
def key_rank_one_round(nr, net,c_num,sub_num):
    sub_keys_rank=np.array([[0.0]]*sub_num)
    #随机生成主密钥
    master_keys = np.frombuffer(urandom(1 * 80), dtype=np.uint8).reshape(-1, 80) & 1
    #生成密钥的十进制
    subkeyss=expand_key(master_keys, nr)
    subkeyss_round = subkeyss[nr]
    Str=''
    for i in subkeyss_round[0]:
        Str+=str(i)
    subkeyss_key_10=int(Str,2) #密钥十进制
    #生成局部密钥范围
    best_key = random.randint(0, sub_num-1)  # 生成1到100之间的整数
    left_key=0
    if subkeyss_key_10>best_key:
        left_key=subkeyss_key_10-best_key
    else:
        left_key=subkeyss_key_10
        best_key=0
    right_key=sub_num-best_key+subkeyss_key_10
    for i in range(c_num):
        #训练生成2**16个数据，预测一下然后相加
        X=dec_on_subkeys(subkeyss,left_key,right_key)
        Z = net.predict(X,batch_size=2000,verbose=0)
        Z = Z/(1+0.000001-Z)
        Z = np.log2(Z)
        sub_keys_rank+=Z
    sub_keys_b = np.arange(left_key,right_key)
    rank_best=np.argmax(sub_keys_rank)
    # rank0 = np.sum(sub_keys_rank > sub_keys_rank[best_key]); 
    rank1 = np.sum(sub_keys_rank >= sub_keys_rank[best_key])
    return(sub_keys_b[rank_best],subkeyss_key_10, rank1)
def stats_key_rank(n, nr, net):
    print("正在进行密钥恢复测试")
    r = np.zeros(n)
    A_SUM=np.array([0]*10)
    mistake_bit=np.array([0]*32)
    mistake_bit_num=np.array([0]*n)
    for i in tqdm(range(n)):
        #a:预测的密钥，b:正确的密钥
        a,b,c = key_rank_one_round(nr, net,4,2**16)
        print("{}".format(c))
        A_SUM[c-1]+=1
        bin_str=bin(np.bitwise_xor(a,b))[2:]
        test_result='0'*(32-len(bin_str))+bin_str
        test_result=[int(bit) for bit in test_result]
        mistake_bit_num[i]=np.sum(test_result)
        mistake_bit=np.sum((mistake_bit,test_result),axis=0)
    return(mistake_bit_num, mistake_bit, A_SUM)
model = make_resnet(depth=10)
# model = load_model('address of the loaded model')
model.load_weights('input/recover-test/32bit_present.h5')
a,b,r = stats_key_rank(100, 6,model)
print(r)
print(a)
print(b)